In [1]:
# Setup OpenAI Agent
import openai
openai.api_key = 'sk-your-key'
from llama_index import VectorStoreIndex, SimpleDirectoryReader


In [2]:
# Create the Agent with access to our tools
documents = SimpleDirectoryReader('llama_hub/tools').load_data()
index = VectorStoreIndex.from_documents(documents)
agent = index.as_chat_engine()
agent.chat("""
    You are a specialized agent will help me create descriptions of Tools that are informative and instruct a user how to use the tools.
    A tool is simply a function that will later be made callable by large language models like yourself.
    Generally tool prompts will describe the function of the tool, and provide information on the arguments and return values.
    
    Here is an example of a function definition and a useful tool prompt:
    def generate_image_variation(self, url: str, n: Optional[int] = 1, size: Optional[str] = '256x256') -> str:
        ```Accepts the url of an image and uses OpenAIs api to generate a variation of the image.
        This tool can take smaller images and create higher resolution variations, or vice versa.

        When passing a url from "generate_images" ALWAYS pass the url exactly as it was returned from the function, including ALL query parameters
        args:
            url (str): The url of the image to create a variation of
            n (Optional[int]): The number of images to generate. Defaults to 1.
            size (Optional[str]): The size of the image(s) to generate. Defaults to 256x256. Other accepted values are 1024x1024 and 512x512
        ```
        
    When the user gives you functions, generate a tool prompt and signature like above
    To assist you, a directory of example tool implmentations has been loaded
    
    The first argument to all tools is self, as they are defined in a class
""")

AgentChatResponse(response='This tool accepts a URL string as an argument and returns a string. It also takes two optional arguments, an integer (n) and a string (size). The function generates n variations of an image from the given URL, with the size of the image specified by the size argument.', sources=[])

In [3]:
print(agent.chat("""
write a tool prompt and typed function definition for dot(a, b) taking two lists of ints and calculating the dot product of two array
"""))
print(agent.chat("""
write a tool prompt and typed function definition for inner(a, b) taking two lists of ints calculating the inner product of two arrays.
"""))
print(agent.chat("""
write a tool prompt and typed function definition for outer(a, b) taking two lists of ints calculating the outer product of two arrays.
"""))
print(agent.chat("""
write a tool prompt and typed function definition for eig(a) computing the eigenvalues and right eigenvectors of a square array.
"""))


This tool takes two lists of integers and calculates the dot product of the two arrays.

```
def dot(self, a: List[int], b: List[int]) -> int:
    """Calculates the dot product of two arrays.
    
    Args:
        a (List[int]): The first array.
        b (List[int]): The second array.
    
    Returns:
        int: The dot product of the two arrays.
    """
```

This tool takes two lists of integers and calculates the inner product of the two arrays.

```
def inner(self, a: List[int], b: List[int]) -> int:
    """Calculates the inner product of two arrays.
    
    Args:
        a (List[int]): The first array.
        b (List[int]): The second array.
    
    Returns:
        int: The inner product of the two arrays.
    """
```

This tool takes two lists of integers and calculates the outer product of the two arrays.

```
def outer(self, a: List[int], b: List[int]) -> List[List[int]]:
    """Calculates the outer product of two arrays.
    
    Args:
        a (List[int]): The first

In [4]:
"""Numpy tool spec."""

from llama_index.tools.tool_spec.base import BaseToolSpec
from typing import Optional, List, Tuple
import numpy as np

class NumpyToolSpec(BaseToolSpec):
    """Numpy Tool Spec"""

    spec_functions = ["dot", "inner", "outer", "eig"]

    def dot(self, a: List[int], b: List[int]) -> int:
        """Calculates the dot product of two arrays.

        Args:
            a (List[int]): The first array.
            b (List[int]): The second array.

        Returns:
            int: The dot product of the two arrays.
        """
        return np.dot(np.array(a), np.array(b))


    def inner(self, a: List[int], b: List[int]) -> int:
        """Calculates the inner product of two arrays.

        Args:
            a (List[int]): The first array.
            b (List[int]): The second array.

        Returns:
            int: The inner product of the two arrays.
        """
        return np.inner(np.array(a), np.array(b))


    def outer(self, a: List[int], b: List[int]) -> List[List[int]]:
        """Calculates the outer product of two arrays.

        Args:
            a (List[int]): The first array.
            b (List[int]): The second array.

        Returns:
            List[List[int]]: The outer product of the two arrays.
        """
        return np.outer(np.array(a), np.array(b))

    def eig(self, a: List[List[int]]) -> Tuple[List[float], List[List[float]]]:
        """Computes the eigenvalues and right eigenvectors of a square array.

        Args:
            a (List[List[int]]): The square array.

        Returns:
            Tuple[List[float], List[List[float]]]: A tuple containing the eigenvalues and right eigenvectors of the array.
        """
        return np.linalg.eig(np.array(a))

In [5]:
from llama_index.agent import OpenAIAgent
agent = OpenAIAgent.from_tools(NumpyToolSpec().to_tool_list(), verbose=True)

In [6]:
print(agent.chat('Calculate the dot, inner and outer product of [1,2,5,1] and [3,5,1,2], then the eigenvalues for the outer product'))


=== Calling Function ===
Calling function: dot with args: {
  "a": [1, 2, 5, 1],
  "b": [3, 5, 1, 2]
}
Got output: 20
=== Calling Function ===
Calling function: inner with args: {
  "a": [1, 2, 5, 1],
  "b": [3, 5, 1, 2]
}
Got output: 20
=== Calling Function ===
Calling function: outer with args: {
  "a": [1, 2, 5, 1],
  "b": [3, 5, 1, 2]
}
Got output: [[ 3  5  1  2]
 [ 6 10  2  4]
 [15 25  5 10]
 [ 3  5  1  2]]
=== Calling Function ===
Calling function: eig with args: {
  "a": [[ 3,  5,  1,  2],
        [ 6, 10,  2,  4],
        [15, 25,  5, 10],
        [ 3,  5,  1,  2]]
}
Got output: (array([ 0.00000000e+00,  2.00000000e+01, -5.83778177e-16, -7.75304324e-17]), array([[-0.71902339, -0.1796053 ,  0.03845666, -0.72391687],
       [ 0.25377296, -0.3592106 , -0.17456401,  0.28698605],
       [ 0.6344324 , -0.89802651,  0.97771147, -0.33013148],
       [ 0.12688648, -0.1796053 , -0.11013071,  0.53347593]]))
The dot product of [1, 2, 5, 1] and [3, 5, 1, 2] is 20.

The inner product of [1, 